Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil

import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'
Reference_Data_database = 'CLE_Reference_Data'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"
external_data = home_folder + "\external_data"


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
os.path.abspath(os.getcwd())

2024-08-14 17:57:05.950433
svmem(total=16756752384, available=4684972032, percent=72.0, used=12071780352, free=4684972032)


'C:\\Users\\m033\\OneDrive - GWLE\\Documents\\GitHub\\Data-Science-Thesis'

# 1. Get supplementary client features.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
# READ IN directly from DW
def customer_data(server, database):
    conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+server+';'
         'DATABASE='+database+';'
         'Trusted_Connection=yes;')
  
    query = """SELECT pol.Policy_Number
      ,pol.Risk_Commencement_Date AS RiskCommDt
--      ,pol.Cover_End_Date AS CoverEndDt
      ,pa.VIP_Type_Id          AS VIPType
      ,pa.Nationality_Code     AS Nationality
      ,CASE WHEN pa.Is_Smoker = 'True' THEN 1
            ELSE 0 
       END as IsSmoker
      ,pa.Politically_Exposed_Id AS PoliticallyExposed
      ,pa.Birth_Place               As BirthPlace
      ,a.Postcode                   AS AddressPostcode
	  ,pa.Sex_Id                    AS Gender
	  ,pa.Marital_Status_Id         AS MaritalStatus
	  ,pla.Pension_Disability_Level AS PensionDisability
	  ,pla.WOP_Disability_Level     AS WOPDisability
      ,a.City                       AS AddressCity
	  ,pa.Date_Of_Birth             AS DOB
--      ,DATEDIFF(month, GETDATE(),pol.Cover_End_Date) AS TermRemainingMths
--      ,DATEDIFF(month, GETDATE(),pol.Cover_End_Date) AS TermRemainingMths
      ,pa.Partner_Type_De           AS PartnerType
      ,pa.Is_EU_Sanctioned          AS EUSanctioned
      ,a.District                   As AddressDistrict
FROM CLT_Policy pol,
		 CLT_Product prd,
		 CLT_Policy_Life_Assured pla,
	  	 CLT_Partner pa,
		 CLT_Address a
WHERE prd.Product_Id = pol.Product_Id
AND   pla.Policy_Id = pol.Policy_Id
AND   pol.Policy_id = pla.Policy_Id
AND   pla.Partner_Id = pa.Partner_Id
AND   pa.Partner_Number = a.Partner_Number
AND   prd.Product_Code IN ('FKP_','GNPP','GNRP','XRPP');"""    
    
    
    return pd.read_sql(query, conn)

df_customer = customer_data(SQL_server, Reporting_database)

# Types and row count
# print stats.
print(len(df_customer.index))
now = datetime.now()
print(now)
print('==========================')

print(df_customer.dtypes)

# No displays to protect PII data
#df_customer.head(5)

137724
2024-08-14 17:57:10.027516
Policy_Number          object
RiskCommDt             object
VIPType                 int64
Nationality            object
IsSmoker                int64
PoliticallyExposed      int64
BirthPlace             object
AddressPostcode        object
Gender                  int64
MaritalStatus           int64
PensionDisability     float64
WOPDisability         float64
AddressCity            object
DOB                    object
PartnerType            object
EUSanctioned             bool
AddressDistrict        object
dtype: object


## Drop obsolete or repeated features.

## Check for nulls

In [4]:
df_customer.dtypes

Policy_Number          object
RiskCommDt             object
VIPType                 int64
Nationality            object
IsSmoker                int64
PoliticallyExposed      int64
BirthPlace             object
AddressPostcode        object
Gender                  int64
MaritalStatus           int64
PensionDisability     float64
WOPDisability         float64
AddressCity            object
DOB                    object
PartnerType            object
EUSanctioned             bool
AddressDistrict        object
dtype: object

In [5]:
df_customer.isna().any()

Policy_Number         False
RiskCommDt            False
VIPType               False
Nationality           False
IsSmoker              False
PoliticallyExposed    False
BirthPlace            False
AddressPostcode       False
Gender                False
MaritalStatus         False
PensionDisability      True
WOPDisability          True
AddressCity           False
DOB                    True
PartnerType           False
EUSanctioned          False
AddressDistrict       False
dtype: bool

## Replace any nulls before checking the distributions of values for each column.

In [6]:
df_customer['PensionDisability'] = df_customer['PensionDisability'].fillna(0).apply(np.int64)
df_customer['WOPDisability'] = df_customer['WOPDisability'].fillna(0).apply(np.int64)


## Set the Postcode to int to facilitate the join.

In [7]:
df_customer['AddressPostcode'] = (
    pd.to_numeric(df_customer['AddressPostcode'],
                  errors='coerce')
      .fillna(0)
    )
df_customer['AddressPostcode'] = df_customer['AddressPostcode'].astype(int)
df_customer.dtypes

Policy_Number         object
RiskCommDt            object
VIPType                int64
Nationality           object
IsSmoker               int64
PoliticallyExposed     int64
BirthPlace            object
AddressPostcode        int32
Gender                 int64
MaritalStatus          int64
PensionDisability      int64
WOPDisability          int64
AddressCity           object
DOB                   object
PartnerType           object
EUSanctioned            bool
AddressDistrict       object
dtype: object

# Read in the postcode data.

In [8]:
# giving directory name
filename = external_data + '\Postcode_State_Metadata.xlsx'

#df_Postcode = pd.read_excel(filename,sheet_name='Tabelle1')
df_Postcode = pd.read_csv(filename)
df_Postcode['Postal_Code'] = df_Postcode['Postal_Code'].astype(int)

print(df_Postcode.dtypes)

# No displays to protect PII data
#df_Postcode.head(2)

Postal_Code          int32
Postal_Code_Name    object
District            object
State               object
dtype: object


# Merge the address and the external postcode data.

In [9]:
# Left Join the list of campaign contract numbers to the main dataset.
df_customer = pd.merge(df_customer, df_Postcode, left_on = ['AddressPostcode'], right_on = ['Postal_Code'] , how = 'left')

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_customer.head(5)

2024-08-14 17:57:11.112712
svmem(total=16756752384, available=4574580736, percent=72.7, used=12182171648, free=4574580736)


# Merge in the sinus milieus data.

In [10]:
# giving directory name
filename = external_data + '\sinus_milieu.csv'


df_Sinus_Milieus = pd.read_csv(filename)

print(df_Sinus_Milieus.dtypes)

# No displays to protect PII data
#df_Sinus_Milieus.head(2)

CONTRACT_NUMBER                 object
DATA_VERSION                   float64
POLICY_NUMBER                   object
VN_MILIEU_CODE                  object
VP_MILIEU_CODE                  object
VN_TARGET_GROUP_CODE            object
VP_TARGET_GROUP_CODE            object
VN_MILIEU_DESCRIPTION           object
VN_TARGET_GROUP_DESCRIPTION     object
VP_MILIEU_DESCRIPTION           object
VP_TARGET_GROUP_DESCRIPTION     object
dtype: object


# Merge in the Sinus data.

In [11]:
df_customer = pd.merge(df_customer, df_Sinus_Milieus, left_on = ['Policy_Number'], right_on = ['CONTRACT_NUMBER'] , how = 'left') 

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_customer.dtypes)


# No displays to protect PII data
#df_customer.head(5)

2024-08-14 17:57:11.976745
svmem(total=16756752384, available=4552318976, percent=72.8, used=12204433408, free=4552318976)
Policy_Number                   object
RiskCommDt                      object
VIPType                          int64
Nationality                     object
IsSmoker                         int64
PoliticallyExposed               int64
BirthPlace                      object
AddressPostcode                  int32
Gender                           int64
MaritalStatus                    int64
PensionDisability                int64
WOPDisability                    int64
AddressCity                     object
DOB                             object
PartnerType                     object
EUSanctioned                      bool
AddressDistrict                 object
Postal_Code                    float64
Postal_Code_Name                object
District                        object
State                           object
CONTRACT_NUMBER                 object
DATA_VERSION       

## Drop all non predictive features and write to Excel

In [12]:
df_customer = df_customer.drop(['AddressPostcode','CONTRACT_NUMBER'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Client_data.csv'
df_customer.to_csv(filename, encoding='utf-8', index=False)        

# No displays to protect PII data
#df_customer.head(10)

C:\Users\m033\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:2106: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [13]:
%who_ls DataFrame 

['df_Postcode', 'df_Sinus_Milieus', 'df_customer']

In [14]:
del df_Postcode
del df_Sinus_Milieus
del df_customer



In [15]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:57:14.680309
svmem(total=16756752384, available=4627718144, percent=72.4, used=12129034240, free=4627718144)
